<a href="https://colab.research.google.com/github/Dhananjay-97/notebooks/blob/main/rag/langchain-retrieval-augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-augmentation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-augmentation.ipynb)

# Retrieval-Augmented Generation with Pinecone, LangChain and OpenAI

## Fixing LLMs that Hallucinate

In this notebook, you'll learn one of the most common applications of retrieval-augmented generation: giving large language models access to up-to-date information.


### Demo Data: Pinecone Documentation

A great example to use RAG is when augmenting LLMs with information that may not exist in their training data. This could private data, internal company information, or data that has been updated post a training cutoff. In our case, many modern LLMs are rained on Pinecone data that has since been updated, such as release notes, quickstart guides, blog posts, docs, and code.

In this example, we'll show the differences in generation from OpenAI's LLMs when asked about Pinecone's release notes! We'll orchestrate our RAG workflow using LangChain, a popular framework for AI applications.

In [1]:
!pip install -qU \
    "langchain[openai]"\
    langchain-text-splitters==0.3.8 \
    langchain-pinecone==0.2.1 \
    pinecone-notebooks==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0

---

## Building a Knowledge Base with our Vector Database

Building more reliable LLMs tools requires an external _"Knowledge Base"_, a database that we can query and update periodically with information.

Specifically, we will need to retrieve information that is relevant to our queries. To do this we need to use _"dense vector embeddings"_. These can be thought of as numerical representations of the *meaning* behind our sentences.

There are many options for creating these dense vectors, like open source [sentence transformers embedding models](https://www.pinecone.io/learn/series/nlp/) or OpenAI's [text-embedding-3-small model](https://platform.openai.com/docs/models/text-embedding-3-small). We will use OpenAI's offering in this example.

### Before you begin...

Be sure to grab a [free Pinecone account](https://app.pinecone.io/?sessionType=signup) and an OpenAI API key, [located here](https://platform.openai.com/api-keys)!

In [2]:
## Getting our Dataset:

# These are markdown versions of our release notes from 2025 and 2024
release_notes_2025 = "https://docs.pinecone.io/release-notes/2025.md"
release_notes_2024 = "https://docs.pinecone.io/release-notes/2024.md"


## Preprocessing our data

We'll use Requests and LangChain to pull down the release notes, and process the associated Markdown. We've used splitters that correspond to the release year, month/year, and features.
This will take us from raw Markdown files to LangChain Documents, which we'll embed and store in Pinecone.

In [3]:
# We'll grab these urls and parse them using LangChain's textsplitter for markdown
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.documents import Document
import requests

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "release"), ("##", "month_year"), ("###", "feature")])

def download_link(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text


def add_document_metadata(doc, new_metadata):
    # returns new documents with updated metadata
    old_metadata = doc.metadata
    new_metadata = {**old_metadata, **new_metadata}
    return Document(page_content=doc.page_content, metadata=new_metadata)


def preprocess_pinecone_docs(urls):

    pinecone_docs = []
    for url in urls:
        # download the markdown
        response = download_link(url)
        split_text = splitter.split_text(response)
        # Update metadata to include url as source
        split_text = [add_document_metadata(doc, {"source": url, "chunk_num": num}) for num, doc in enumerate(split_text)]
        pinecone_docs.extend(split_text)
    return pinecone_docs


pinecone_docs = preprocess_pinecone_docs([release_notes_2024,release_notes_2025])

Let's take a closer look at one of these notes

In [4]:
print("Document content: ", pinecone_docs[2].page_content)
print("Document metadata: ", pinecone_docs[2].metadata)

Document content:  Pinecone Assistant can now [return a JSON response](/guides/assistant/chat-with-assistant#json-response).  
***  
You can now [create an assistant](/reference/api/2025-01/assistant/create_assistant) in the `eu` region.
</Update>  
<Update label="2024-12-17" tags={["Database"]}>
Document metadata:  {'release': '2024 releases', 'month_year': 'December 2024', 'feature': 'Pinecone Assistant JSON mode and EU region deployment', 'source': 'https://docs.pinecone.io/release-notes/2024.md', 'chunk_num': 2}


In [5]:
print("Document content: ", pinecone_docs[-1].page_content)
print("Document metadata: ", pinecone_docs[-1].metadata)

Document content:  Released [`v2.2.0`](https://github.com/pinecone-io/go-pinecone/releases/tag/v2.2.0) of the [Pinecone Go SDK](/reference/go-sdk). This version adds support for [index tags](/guides/manage-data/manage-indexes#configure-index-tags) when creating or configuring indexes.
</Update>
Document metadata:  {'release': '2025 releases', 'month_year': 'January 2025', 'feature': 'Released Go SDK v2.2.0', 'source': 'https://docs.pinecone.io/release-notes/2025.md', 'chunk_num': 53}


## Setting up Pinecone


Next, we'll setup our API keys. For notebooks in Colab environments, we've included a handy block that helps set a Pinecone API key in your environment. In all other contexts, it's sufficient to save your Pinecone and OpenAI keys in your local environment.

Run the next two blocks and enter your Pinecone and OpenAI keys as needed:

In [6]:

import os
from getpass import getpass

def get_pinecone_api_key():
    """
    Get Pinecone API key from environment variable or prompt user for input.
    Returns the API key as a string.

    Only necessary for notebooks. When using Pinecone yourself,
    you can use environment variables or the like to set your API key.
    """
    api_key = os.environ.get("PINECONE_API_KEY")

    if api_key is None:
        try:
            # Try Colab authentication if available
            from pinecone_notebooks.colab import Authenticate
            Authenticate()
            # If successful, key will now be in environment
            api_key = os.environ.get("PINECONE_API_KEY")
        except ImportError:
            # If not in Colab or authentication fails, prompt user for API key
            print("Pinecone API key not found in environment.")
            api_key = getpass("Please enter your Pinecone API key: ")
            # Save to environment for future use in session
            os.environ["PINECONE_API_KEY"] = api_key

    return api_key

PINECONE_API_KEY = get_pinecone_api_key()


## Setup OpenAI API Key



In [7]:
def get_openai_api_key():
    """
    Get OpenAI API key from environment variable or prompt user for input.
    Returns the API key as a string.
    """

    api_key = os.environ.get("OPENAI_API_KEY")

    if api_key is None:
        try:
            api_key = getpass("Please enter your OpenAI API key: ")
            # Save to environment for future use in session
            os.environ["OPENAI_API_KEY"] = api_key
        except Exception as e:
            print(f"Error getting OpenAI API key: {e}")
            return None

    return api_key

In [8]:
OPENAI_API_KEY = get_openai_api_key()

Please enter your OpenAI API key: ··········


## Setting up Pinecone Index
We'll instantiate a Pinecone client, and create an index with a few key properties:
- index_name, which identifies our index
- dimension, which corresponds to the OpenAI embedding model vector size we'll use
- metric, which corresponds to the way "closeness" is evaluated with our vectors
- a spec, which determines the kind of index we are setting up. In this case, it's a free tier Pinecone serverless index

In [10]:
from pinecone import Pinecone

pc = Pinecone(
        api_key="pcsk_4LYjYn_N1TfoeLwfM1Yp6hHRaCZjXrBhpUk359ZGEG3VBeeBPDKPzZubPSYRYFtDCUveTP",
        # You can remove this parameterfor your own projects
        source_tag="pinecone_examples:docs:langchain_retrieval_augmentation"
    )


In [11]:
index_name = "langchain-pinecone-rag"
from pinecone import ServerlessSpec

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        # dimension of the vector embeddings produced by OpenAI's text-embedding-3-small
        dimension=1536,
        metric="cosine",
        # parameters for the free tier index
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Initialize index client
index = pc.Index(name=index_name)

# View index stats
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Embedding our documents and upserting into Pinecone

Next, we'll setup our OpenAI embedding model and Pinecone vector database within LangChain. To do this, we import the related abstractions from LangChain and pass in our API keys and model names.

After that, we generate ids for each document-chunk to better manage them within Pinecone.

In [12]:
[doc.metadata for doc in pinecone_docs]

[{'release': '2024 releases',
  'month_year': 'December 2024',
  'source': 'https://docs.pinecone.io/release-notes/2024.md',
  'chunk_num': 0},
 {'release': '2024 releases',
  'month_year': 'December 2024',
  'feature': 'Increased namespaces limit',
  'source': 'https://docs.pinecone.io/release-notes/2024.md',
  'chunk_num': 1},
 {'release': '2024 releases',
  'month_year': 'December 2024',
  'feature': 'Pinecone Assistant JSON mode and EU region deployment',
  'source': 'https://docs.pinecone.io/release-notes/2024.md',
  'chunk_num': 2},
 {'release': '2024 releases',
  'month_year': 'December 2024',
  'feature': 'Released Spark-Pinecone connector v1.2.0',
  'source': 'https://docs.pinecone.io/release-notes/2024.md',
  'chunk_num': 3},
 {'release': '2024 releases',
  'month_year': 'December 2024',
  'feature': 'New integration with HoneyHive',
  'source': 'https://docs.pinecone.io/release-notes/2024.md',
  'chunk_num': 4},
 {'release': '2024 releases',
  'month_year': 'December 2024',


In [13]:

from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

vector_store = PineconeVectorStore(index=index, embedding=embeddings)


# do url_title, chunk_num to enable subscriptable hashing and replacement

def clean_url_for_title(url):
    # grabs the end of the url minus .md
    return url.split("/")[-1].replace(".md", "")

# Here, we follow a schema that puts the document name, and the chunk number together, like doc1#chunk1

def generate_ids(doc_chunk):

    title = clean_url_for_title(doc_chunk.metadata['source'])
    chunk_num = doc_chunk.metadata['chunk_num']
    feature = doc_chunk.metadata['feature'] if 'feature' in doc_chunk.metadata else "na"
    return f"release_{title}#feature_{feature}#chunk_num{chunk_num}"

ids = [generate_ids(doc) for doc in pinecone_docs]


# To learn more, look here: https://docs.pinecone.io/guides/manage-data/manage-document-chunks

vector_store.add_documents(documents=pinecone_docs, ids=ids)


['release_2024#feature_na#chunk_num0',
 'release_2024#feature_Increased namespaces limit#chunk_num1',
 'release_2024#feature_Pinecone Assistant JSON mode and EU region deployment#chunk_num2',
 'release_2024#feature_Released Spark-Pinecone connector v1.2.0#chunk_num3',
 'release_2024#feature_New integration with HoneyHive#chunk_num4',
 'release_2024#feature_Released Python SDK v5.4.2#chunk_num5',
 'release_2024#feature_Launch week: Pinecone Local#chunk_num6',
 'release_2024#feature_Launch week: Enhanced security and access controls#chunk_num7',
 'release_2024#feature_Launch week: `pinecone-rerank-v0` and `cohere-rerank-3.5` on Pinecone Inference#chunk_num8',
 'release_2024#feature_Launch week: Integrated Inference#chunk_num9',
 'release_2024#feature_Released .NET SDK v2.1.0#chunk_num10',
 'release_2024#feature_Improved batch deletion guidance#chunk_num11',
 'release_2024#feature_Launch week: Released `pinecone-sparse-english-v0`#chunk_num12',
 'release_2024#feature_na#chunk_num13',
 're

## Bringing it all together: Using OpenAI to learn about our releases

Finally, we'll setup an OpenAI LLM endpoint to generate responses given a user's query about our release notes!

In [14]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [15]:
#OpenAI models will be unable to answer this due to training cutoffs in 2023-2024

query = "Tell me about version 7.0 of the Pinecone Python SDK"

retrieved_docs = vector_store.similarity_search(query, k=5)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

prompt = f'''You are an assistant that answers questions exclusively about the
Pinecone SDK release notes:

Here's a question: {query}

Here's some context from the release notes:

{docs_content}


Question: {query}

Answer:
'''

# This will take a few seconds to run, due to the generation of the response from OpenAI
answer = llm.invoke(prompt)

We can peek at the retrieved documents to confirm up to date information is being passed in:

In [16]:
for num, d in enumerate(retrieved_docs):
    print(f"Doc number: {num+1}")
    print(d.page_content)
    print("Metadata:")
    print(d.metadata)
    print("-"*100)


### Comparing responses with/without RAG

Without our RAG pipeline with release notes indexed, OpenAI models will be
unable to answer questions about new versions of Pinecone. They may even "hallucinate", or fabricate information about the versions that may not exist!

In [17]:
print(llm.invoke("Tell about the most recent major feature release in the Pinecone Python SDK").content)

As of my last knowledge update in October 2023, I don't have the specific details about the most recent major feature release of the Pinecone Python SDK. However, to find the latest information regarding updates or releases, it is best to refer to the official Pinecone documentation or their GitHub repository. These resources typically include release notes that detail new features, improvements, and bug fixes.

If you're looking for information on a particular feature or change, or if there's something specific about the SDK you're curious about, feel free to ask!


However, with our pipeline in place, we get an answer that is more likely to be correct, and definitely grounded.

In [18]:
print(answer.content)

Version 7.0 of the Pinecone Python SDK introduced several noteworthy features and improvements. Key highlights include:

1. **New Indexing and Querying API**: The SDK now supports a more streamlined and efficient way to create, update, and query indices, making it easier for users to manage their vector data.

2. **Enhanced Performance**: Various performance optimizations have been implemented to improve the speed and efficiency of operations within the SDK.

3. **Improved Documentation**: The release includes updated and clearer documentation, making it easier for users to understand how to utilize the SDK effectively.

4. **Compatibility Updates**: Version 7.0 includes updates to ensure compatibility with the latest versions of Python and other dependencies.

5. **Bug Fixes and Stability Improvements**: This version also addresses several bugs from previous versions, enhancing overall stability and user experience.

This release marks a significant step forward for developers working

## Wrapping up

And that's that! You've successfully implemented retrieval augmented generation with Pinecone, OpenAI and LangChain. Wanna learn more? Try implementing the following:

- Sparse search to enable precise time, date and feature recognition in query results
- Expanding the set of documents to encompass all Pinecone documentaiotn
- Learning how to chunk and process code data, to build your own code assistant

To finish, let's delete our index:

In [19]:
pc.delete_index(name=index_name)

---